## Network Traffic Pipeline Deployment in UCS

### Clone git repo

In [1]:
! git clone https://github.com/CiscoAI/cisco-kubeflow-starter-pack.git

Cloning into 'cisco-kubeflow-starter-pack'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 134 (delta 15), reused 125 (delta 14), pack-reused 0
Receiving objects: 100% (134/134), 1.69 MiB | 13.94 MiB/s, done.
Resolving deltas: 100% (15/15), done.


### Import libraries

In [11]:
import kfp
import os
from kubernetes import client
import calendar
import time

### Component files Declarations

In [12]:
path='cisco-kubeflow-starter-pack/apps/networking/network-traffic/onprem/pipelines/'
component_root_train= path+'components/v2/tf-model-train/'
component_root_serve = path+'components/v2/tf-serving-deploy-service/'
component_root_webui = path+'components/v2/tf-webui-deploy-service/'
component_root_tensorboard= path+'components/v2/tf-tensorboard-deploy-service/'

#### Components Description

tf_train_model_op &nbsp;&nbsp;&nbsp; - &nbsp; Load dataset from nfs-volume, train Network Traffic model  and save model in nfs-volume. <br> 
tf_serve_model_op &nbsp; - &nbsp; Load Network Traffic model from nfs-volume and serve the model using tf-serving for later prediction. <br> 
tf_webui_op &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - &nbsp; Web application client UI loads Unlabeled data, does preprocessing, send API request to serving and display Predicted Location. <br> 
tf_tensor_op &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  - &nbsp; Tensorboard UI which provides the visualization metrics such as loss and accuracy, model graph (ops and layers) and histograms of weights, biases, or other tensors for Network Traffic Model <br> 

### Load components from respective .YAML config files

In [13]:
tf_train_model_op = kfp.components.load_component_from_file(os.path.join(component_root_train, 'component.yaml')) 
tf_serve_model_op = kfp.components.load_component_from_file(os.path.join(component_root_serve, 'component.yaml'))
tf_webui_op = kfp.components.load_component_from_file(os.path.join(component_root_webui, 'component.yaml'))
tf_tensor_op = kfp.components.load_component_from_file(os.path.join(component_root_tensorboard, 'component.yaml'))

### Define Volume and Volume Mounts

In [14]:
nfs_pvc = client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs')
nfs_volume = client.V1Volume(name='nfs', persistent_volume_claim=nfs_pvc)
nfs_volume_mount = client.V1VolumeMount(mount_path='/mnt/', name='nfs')

### Define pipeline function

In [15]:
# Creating timestamp
timestamp = str(calendar.timegm(time.gmtime()))
print(timestamp)
def network_pipeline():
        
#     Defining Task for Model training
    tf_train_model_task = tf_train_model_op()
    tf_train_model_task.add_volume(nfs_volume)
    tf_train_model_task.add_volume_mount(nfs_volume_mount)
    
    #Defining Task to perform serving    
    tf_serve_model_task = tf_serve_model_op(timestamp=timestamp)
    tf_serve_model_task.add_volume(nfs_volume)
    tf_serve_model_task.add_volume_mount(nfs_volume_mount)
    tf_serve_model_task.after(tf_train_model_task)

#     Defining task for tensorboard creation    
    tf_tensorboard_task = tf_tensor_op(timestamp=timestamp)
    tf_tensorboard_task.add_volume(nfs_volume)
    tf_tensorboard_task.add_volume_mount(nfs_volume_mount)
    tf_tensorboard_task.after(tf_train_model_task)
    
    
    #Defining task for webUI creation
    tf_webui_task = tf_webui_op(image_path="docker.io/edward1723/kf-network-webui-app:0.1",
                                timestamp=timestamp)
    tf_webui_task.add_volume(nfs_volume)
    tf_webui_task.add_volume_mount(nfs_volume_mount)
    tf_webui_task.after(tf_serve_model_task)
    
#Creating a pipeline run
kfp.Client().create_run_from_pipeline_func(network_pipeline, arguments={})

1584448738


RunPipelineResult(run_id=d82a74dc-82c8-433a-9151-cd0ad4d38b79)

### Before accessing service URLs below, make sure that Pipeline Run is Complete in the Dashboard

### Get Tensorboard URL

In [17]:
TB_SERVICE_NAME = "tensorboard-"+str(timestamp)+"-network-service"
port = !(kubectl get svc {TB_SERVICE_NAME} -n kubeflow -o yaml | grep -i nodePort: | cut -d":" -f 2)
print('Tensorboard service is available at - http://INGRESS_IP:{}'.format(port[0].lstrip()))

Tensorboard service is available at - http://INGRESS_IP:30042


### Get WebUI URL 
And upload csv from https://cse-cic-ids2018.s3.ca-central-1.amazonaws.com/Processed+Traffic+Data+for+ML+Algorithms/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv

In [18]:
WEBUI_SERVICE_NAME = "webui-"+str(timestamp)+"-network-service"
port = !(kubectl get svc {WEBUI_SERVICE_NAME} -n kubeflow -o yaml | grep -i nodePort: | cut -d":" -f 2)
print('WebUI service is available at - http://INGRESS_IP:{}'.format(port[0].lstrip()))

WebUI service is available at - http://INGRESS_IP:31436
